In [4]:
from lib.data import get_c4
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf", use_fast=False)
get_c4(128, 0, 2048, tokenizer)

/home/coder/conda/envs/prune_llm/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Generating train split: 356317 examples [00:43, 8140.28 examples/s]
Generating validation split: 45576 examples [00:05, 8261.37 examples/s]


([(tensor([[  596, 12113,  5593,  ...,  5557,   291,   448]]),
   tensor([[-100, -100, -100,  ..., -100, -100,  448]])),
  (tensor([[29896, 29945, 29889,  ..., 25227,   355, 14321]]),
   tensor([[ -100,  -100,  -100,  ...,  -100,  -100, 14321]])),
  (tensor([[ 3412,   263, 24589,  ..., 18161,  6434, 29889]]),
   tensor([[ -100,  -100,  -100,  ...,  -100,  -100, 29889]])),
  (tensor([[ 2300, 29892, 28944,  ...,   593, 29879,   304]]),
   tensor([[-100, -100, -100,  ..., -100, -100,  304]])),
  (tensor([[10757,   304,  1510,  ...,  2125,   373,   278]]),
   tensor([[-100, -100, -100,  ..., -100, -100,  278]])),
  (tensor([[ 304, 2317,  714,  ..., 5855, 2729,  373]]),
   tensor([[-100, -100, -100,  ..., -100, -100,  373]])),
  (tensor([[ 297,  278, 2737,  ...,  411,  297, 8105]]),
   tensor([[-100, -100, -100,  ..., -100, -100, 8105]])),
  (tensor([[ 2060, 29892, 18987,  ...,   297,   278,   363]]),
   tensor([[-100, -100, -100,  ..., -100, -100,  363]])),
  (tensor([[ 366,  393,  505,  .

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3080 Ti'